In [1]:
pip install yfinance pandas numpy


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import copy
import time

# --- 전략 파라미터 설정 (백테스트 함수와 동일하게 유지) ---
STRATEGY_PARAMS = {
    'using_tickers': ['QQQ', 'TQQQ', 'PSQ', 'SPY'],
    'modes': {
        'Normal': {'QQQ': 0.5, 'TQQQ': 0.5},
        'Defense': {},
        'Aggressive': {'TQQQ': 1.0},
        'Unknown': {},
    },
    'low_ma': 25,
    'center_ma': 200,
    'ma_ticker': 'SPY',
    'ma200_gap_threshold_for_qqq_defense': 0.10,
    'psq_defense_exit_threshold': 0.05,
    'normal_dynamic_leverage': {
        'last_year_qqq_underperform': 2.5,
        'last_year_qqq_outperform': 1.5,
        'last_year_qqq_unknown': 2.0,
        'last_year_qqq_threshold': 0.15,
        'lookback_days': 252,
    },
    'aggressive_dynamic_leverage': {
        'lookback_days': 252 * 3,
        'low_threshold': 0.45,
        'high_threshold': 0.65,
        'high_leverage': 3.0,
        'medium_leverage': 2.5,
        'low_leverage': 2.0,
        'default_leverage': 2.5,
    },
}

def get_historical_data(tickers, period="5y"):
    """
    장기 지표 계산을 위한 과거 데이터를 'Open'(시가) 기준으로 가져옵니다.
    """
    try:
        print("3년 이상 과거 데이터를 가져오는 중... (시간이 걸릴 수 있습니다)")
        data = yf.download(tickers, period=period, progress=False)['Open']
        data.fillna(method='ffill', inplace=True)
        if data.isnull().values.any():
            data.fillna(method='bfill', inplace=True)
            if data.isnull().values.any():
                print("오류: 과거 데이터에 누락된 값이 많아 분석을 진행할 수 없습니다.")
                return None
        return data
    except Exception as e:
        print(f"과거 데이터를 가져오는 중 오류 발생: {e}")
        return None

def get_live_prices(tickers):
    """
    프리마켓/정규장/애프터마켓을 구분하여 모든 티커의 현재 가격을 가져옵니다.
    """
    live_prices = {}
    market_status = "확인 불가"

    print("실시간 가격 조회 중...", end="", flush=True)
    for ticker in tickers:
        try:
            info = yf.Ticker(ticker).info
            state = info.get('marketState')
            price = None
            current_status = "장 마감/대체"

            if state == 'PRE': price = info.get('preMarketPrice'); current_status = "프리마켓"
            elif state == 'REGULAR': price = info.get('regularMarketPrice'); current_status = "정규장"
            elif state == 'POST': price = info.get('postMarketPrice'); current_status = "애프터마켓"
            
            if price is None: price = info.get('regularMarketPrice', info.get('previousClose'))
            if price is None:
                hist = yf.Ticker(ticker).history(period="2d", prepost=True)
                if not hist.empty: price = hist['Close'].iloc[-1]; current_status = "최근 체결가"
            if price is None:
                print(f"\n오류: {ticker}의 실시간 가격을 가져올 수 없습니다."); return None, None
            
            live_prices[ticker] = price
            if ticker == STRATEGY_PARAMS['ma_ticker']: market_status = current_status
            print(".", end="", flush=True)
            time.sleep(0.2)
        except Exception as e:
            print(f"\n오류: {ticker} 가격 조회 실패: {e}"); return None, None
            
    print(" 완료.")
    print(f"\n현재 시장 상태: {market_status}")
    print("실시간 가격:", {ticker: f"{price:.2f}" for ticker, price in live_prices.items()})
    return live_prices, market_status

def determine_mode(today, yesterday):
    """주어진 데이터(하루치)를 바탕으로 전략 모드를 결정합니다."""
    p = STRATEGY_PARAMS
    spy = today[p['ma_ticker']]
    spy_ma_low = today[f'SPY_MA{p["low_ma"]}']
    spy_ma_center = today[f'SPY_MA{p["center_ma"]}']
    spy_max = today['SPY_MAX']
    current_mode = yesterday.get('mode', 'Unknown')
    
    if current_mode == 'Normal':
        if spy < spy_ma_center: return 'Defense'
    elif current_mode == 'Defense':
        if spy > spy_ma_center: return 'Normal'
        if spy_ma_low < spy < spy_ma_center: return 'Aggressive'
    elif current_mode == 'Aggressive':
        if spy < spy_ma_low < spy_ma_center: return 'Defense'
        if abs(spy - spy_max) < 1e-6: return 'Normal'
    else:
        if spy > spy_ma_center: return 'Normal'
        if spy_ma_low < spy < spy_ma_center: return 'Aggressive'
        return 'Defense'
    return current_mode

def get_position_details(mode, today, prev_mode):
    """결정된 모드에 따라 최종 포지션과 설명을 반환합니다."""
    p = STRATEGY_PARAMS
    modes = copy.deepcopy(p['modes'])
    
    # 1. Normal 모드 동적 할당
    dyn_lev = p['normal_dynamic_leverage']
    ### <<< 수정된 부분: 이제 'QQQ_1Y_MA200_Return' 지표를 사용합니다. >>> ###
    qqq_return = today['QQQ_1Y_MA200_Return'] 
    
    if pd.isna(qqq_return): leverage = dyn_lev['last_year_qqq_unknown']
    elif qqq_return < dyn_lev['last_year_qqq_threshold']: leverage = dyn_lev['last_year_qqq_underperform']
    else: leverage = dyn_lev['last_year_qqq_outperform']
    normal_qqq_alloc = (3.0 - leverage) / 2.0
    normal_tqqq_alloc = 1.0 - normal_qqq_alloc
    modes['Normal'] = {'QQQ': normal_qqq_alloc, 'TQQQ': normal_tqqq_alloc}

    # 2. Aggressive 모드 동적 할당
    agg_p = p['aggressive_dynamic_leverage']
    three_year_gap = today['3Y_MA200_Gap']
    agg_leverage = agg_p['default_leverage']
    if not pd.isna(three_year_gap):
        if three_year_gap < agg_p['low_threshold']: agg_leverage = agg_p['high_leverage']
        elif three_year_gap < agg_p['high_threshold']: agg_leverage = agg_p['medium_leverage']
        else: agg_leverage = agg_p['low_leverage']
    agg_tqqq_alloc = (agg_leverage - 1.0) / 2.0
    agg_qqq_alloc = 1.0 - agg_tqqq_alloc
    agg_tqqq_alloc = max(0, min(1.0, agg_tqqq_alloc)); agg_qqq_alloc = 1.0 - agg_tqqq_alloc
    modes['Aggressive'] = {'QQQ': agg_qqq_alloc, 'TQQQ': agg_tqqq_alloc}

    # 3. Defense 모드 동적 할당
    spy = today[p['ma_ticker']]; spy_ma_center = today[f'SPY_MA{p["center_ma"]}']
    in_psq_defense_mode = (prev_mode == 'Normal' and mode == 'Defense')
    if mode == 'Defense':
        if in_psq_defense_mode: modes['Defense'] = {'PSQ': 1.0}
        else:
            gap_cond = spy < spy_ma_center * (1 - p['ma200_gap_threshold_for_qqq_defense'])
            if gap_cond: modes['Defense'] = {'QQQ': 1.0}
            else: modes['Defense'] = {}

    position = modes.get(mode, {})
    if not position: return "현금 100%"
    else:
        position_str = " : ".join([f"{ticker}({val:.1%})" for ticker, val in position.items() if val > 0.001])
        return position_str if position_str else "현금 100%"

def run_market_watch_analysis():
    """사용자 요청 시 실시간 분석을 수행하는 메인 함수"""
    p = STRATEGY_PARAMS
    nl = p['normal_dynamic_leverage']; al = p['aggressive_dynamic_leverage']
    print("\n" + "-"*50)
    print("참고: 현재 설정된 주요 동적 할당 파라미터")
    print("-" * 50)
    ### <<< 수정된 부분: Normal 모드 설명 수정 >>> ###
    print("[Normal 모드 레버리지 조건]")
    print(f"  - 현재 QQQ 가격 vs '1년 전 QQQ의 MA200' 수익률 < {nl['last_year_qqq_threshold']:.0%}: 레버리지 {nl['last_year_qqq_underperform']}x")
    print(f"  - 현재 QQQ 가격 vs '1년 전 QQQ의 MA200' 수익률 >= {nl['last_year_qqq_threshold']:.0%}: 레버리지 {nl['last_year_qqq_outperform']}x")
    print("\n[Aggressive 모드 레버리지 조건]")
    print(f"  - 3년 MA200 Gap < {al['low_threshold']:.0%}: 레버리지 {al['high_leverage']}x 적용")
    print(f"  - {al['low_threshold']:.0%} <= 3년 MA200 Gap < {al['high_threshold']:.0%}: 레버리지 {al['medium_leverage']}x 적용")
    print(f"  - 3년 MA200 Gap >= {al['high_threshold']:.0%}: 레버리지 {al['low_leverage']}x 적용")
    print("-" * 50)

    print("\n과거 데이터를 가져오는 중...")
    hist_data = get_historical_data(p['using_tickers'])
    if hist_data is None: return

    live_prices, market_status = get_live_prices(p['using_tickers'])
    if live_prices is None: return
    
    today_series = pd.Series(live_prices, name=datetime.now())
    hist_data.index = pd.to_datetime(hist_data.index)
    data = pd.concat([hist_data, today_series.to_frame().T])
    data[p['using_tickers']] = data[p['using_tickers']].astype(float)
    
    print("지표를 계산하는 중...")
    data[f'SPY_MA{p["low_ma"]}'] = data['SPY'].rolling(window=p['low_ma']).mean()
    data[f'SPY_MA{p["center_ma"]}'] = data['SPY'].rolling(window=p['center_ma']).mean()
    data['SPY_MAX'] = data['SPY'].cummax()
    
    ### <<< 수정된 부분: Normal 모드 지표 계산 로직 변경 >>> ###
    # QQQ의 200일 이동평균선 계산
    data[f'QQQ_MA{p["center_ma"]}'] = data['QQQ'].rolling(window=p["center_ma"]).mean()
    # 1년 전 QQQ의 MA200 값을 가져옴
    lookback_1y = p['normal_dynamic_leverage']['lookback_days']
    one_year_ago_qqq_ma200 = data[f'QQQ_MA{p["center_ma"]}'].shift(lookback_1y)
    # (현재 QQQ 가격 - 1년 전 QQQ MA200) / (1년 전 QQQ MA200) 계산
    data['QQQ_1Y_MA200_Return'] = (data['QQQ'] - one_year_ago_qqq_ma200) / one_year_ago_qqq_ma200
    
    # Aggressive 모드 지표 계산
    lookback_3y = p['aggressive_dynamic_leverage']['lookback_days']
    three_year_ago_spy_ma200 = data[f'SPY_MA{p["center_ma"]}'].shift(lookback_3y)
    data['3Y_MA200_Gap'] = (data['SPY'] - three_year_ago_spy_ma200) / three_year_ago_spy_ma200

    data = data.dropna()

    if len(data) < 3: print("분석에 필요한 데이터가 부족합니다."); return
        
    today, yesterday, day_before_yesterday = data.iloc[-1].copy(), data.iloc[-2].copy(), data.iloc[-3].copy()
    day_before_yesterday['mode'] = determine_mode(day_before_yesterday, data.iloc[-4].copy() if len(data) > 3 else pd.Series())
    yesterday['mode'] = determine_mode(yesterday, day_before_yesterday)
    today['mode'] = determine_mode(today, yesterday)

    spy, spy_ma_low, spy_ma_center, spy_max = today[p['ma_ticker']], today[f'SPY_MA{p["low_ma"]}'], today[f'SPY_MA{p["center_ma"]}'], today['SPY_MAX']
    qqq_return_1y_ma200 = today['QQQ_1Y_MA200_Return'] # 이름 변경
    three_year_gap = today['3Y_MA200_Gap']
    
    agg_p = p['aggressive_dynamic_leverage']
    agg_cond_desc = "데이터 부족"
    if not pd.isna(three_year_gap):
        if three_year_gap < agg_p['low_threshold']: agg_cond_desc = f"< {agg_p['low_threshold']:.0%} (레버리지 3.0x)"
        elif three_year_gap < agg_p['high_threshold']: agg_cond_desc = f"{agg_p['low_threshold']:.0%}-{agg_p['high_threshold']:.0%} (레버리지 2.5x)"
        else: agg_cond_desc = f"> {agg_p['high_threshold']:.0%} (레버리지 2.0x)"

    normal_p = p['normal_dynamic_leverage']
    normal_cond_desc = "데이터 부족"; normal_lev_desc = f" (레버리지 {normal_p['last_year_qqq_unknown']}x)"
    if not pd.isna(qqq_return_1y_ma200):
        is_underperform = qqq_return_1y_ma200 < normal_p['last_year_qqq_threshold']
        if is_underperform:
            normal_cond_desc = f"< {normal_p['last_year_qqq_threshold']:.0%}"; normal_lev_desc = f" (레버리지 {normal_p['last_year_qqq_underperform']}x)"
        else:
            normal_cond_desc = f">= {normal_p['last_year_qqq_threshold']:.0%}"; normal_lev_desc = f" (레버리지 {normal_p['last_year_qqq_outperform']}x)"

    ### <<< 수정된 부분: conditions 딕셔너리 키와 내용 변경 >>> ###
    conditions = {
        f"SPY > MA{p['center_ma']} (상승장 진입)": spy > spy_ma_center,
        f"MA{p['low_ma']} < SPY < MA{p['center_ma']} (회복장 진입)": spy_ma_low < spy < spy_ma_center,
        f"SPY < MA{p['low_ma']} (하락장 지속)": spy < spy_ma_low,
        "SPY 전고점(MAX) 도달": abs(spy - spy_max) < 1e-6,
        "--- 동적 할당 조건 ---": "---",
        f"Normal 모드 (1년전 QQQ MA200 대비 수익률: {qqq_return_1y_ma200:.2%})": True,
        f"  ㄴ 조건: {normal_cond_desc}{normal_lev_desc}": True,
        f"Aggressive 모드 (3년전 SPY MA200 대비 Gap: {three_year_gap:.2%})": True,
        f"  ㄴ 조건: {agg_cond_desc}": True
    }
    
    prev_position = get_position_details(yesterday['mode'], yesterday, day_before_yesterday.get('mode', 'Unknown'))
    today_position = get_position_details(today['mode'], today, yesterday['mode'])

    print("\n" + "="*65)
    print(f" 마켓워치 분석 결과 (기준 시각: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} | 시장: {market_status})")
    print("="*65)
    print(f"250703-3-4\n")
    print(f"어제 포지션 ({yesterday['mode']}): {prev_position}")
    print(f"오늘 포지션 ({today['mode']}): {today_position}\n")
    print("주요 조건 달성 현황:")
    for desc, status in conditions.items():
        if "---" in desc: print(desc); continue
        if "모드" in desc and "ㄴ" not in desc: print(f"- {desc}")
        elif "ㄴ" in desc: print(f" {desc}")
        else: print(f"- {desc}: {'O' if status else 'X'}")
    print("="*65)

# --- 메인 실행 부분 ---
if __name__ == "__main__":
    print("--- 실시간 전략 분석 마켓워치 (원본 Backtest 로직 기반) ---")
    print("분석을 시작하려면 Enter 키를 누르세요.")
    print("프로그램을 종료하려면 'q' 또는 '종료'를 입력 후 Enter를 누르세요.")
    print("-" * 50)

    while True:
        command = input("명령을 입력하세요 (Enter: 분석, q: 종료): ")
        if command.lower() in ['q', 'quit', '종료', 'exit']: print("프로그램을 종료합니다."); break
        try: run_market_watch_analysis()
        except Exception as e: print(f"\n분석 중 예상치 못한 오류가 발생했습니다: {e}")
        print("\n다음 분석을 위해 대기 중입니다...")

--- 실시간 전략 분석 마켓워치 (원본 Backtest 로직 기반) ---
분석을 시작하려면 Enter 키를 누르세요.
프로그램을 종료하려면 'q' 또는 '종료'를 입력 후 Enter를 누르세요.
--------------------------------------------------

--------------------------------------------------
참고: 현재 설정된 주요 동적 할당 파라미터
--------------------------------------------------
[Normal 모드 레버리지 조건]
  - 현재 QQQ 가격 vs '1년 전 QQQ의 MA200' 수익률 < 15%: 레버리지 2.5x
  - 현재 QQQ 가격 vs '1년 전 QQQ의 MA200' 수익률 >= 15%: 레버리지 1.5x

[Aggressive 모드 레버리지 조건]
  - 3년 MA200 Gap < 45%: 레버리지 3.0x 적용
  - 45% <= 3년 MA200 Gap < 65%: 레버리지 2.5x 적용
  - 3년 MA200 Gap >= 65%: 레버리지 2.0x 적용
--------------------------------------------------

과거 데이터를 가져오는 중...
3년 이상 과거 데이터를 가져오는 중... (시간이 걸릴 수 있습니다)
실시간 가격 조회 중...

C:\Windows\Temp\ipykernel_19644\468671792.py:47: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


.... 완료.

현재 시장 상태: 정규장
실시간 가격: {'QQQ': '568.57', 'TQQQ': '89.98', 'PSQ': '32.84', 'SPY': '637.23'}
지표를 계산하는 중...

 마켓워치 분석 결과 (기준 시각: 2025-07-28 22:55:16 | 시장: 정규장)
250703-3-4

어제 포지션 (Normal): QQQ(75.0%) : TQQQ(25.0%)
오늘 포지션 (Normal): QQQ(75.0%) : TQQQ(25.0%)

주요 조건 달성 현황:
- SPY > MA200 (상승장 진입): O
- MA25 < SPY < MA200 (회복장 진입): X
- SPY < MA25 (하락장 지속): X
- SPY 전고점(MAX) 도달: X
--- 동적 할당 조건 ---
- Normal 모드 (1년전 QQQ MA200 대비 수익률: 34.34%)
   ㄴ 조건: >= 15% (레버리지 1.5x)
- Aggressive 모드 (3년전 SPY MA200 대비 Gap: 54.06%)
   ㄴ 조건: 45%-65% (레버리지 2.5x)

다음 분석을 위해 대기 중입니다...

--------------------------------------------------
참고: 현재 설정된 주요 동적 할당 파라미터
--------------------------------------------------
[Normal 모드 레버리지 조건]
  - 현재 QQQ 가격 vs '1년 전 QQQ의 MA200' 수익률 < 15%: 레버리지 2.5x
  - 현재 QQQ 가격 vs '1년 전 QQQ의 MA200' 수익률 >= 15%: 레버리지 1.5x

[Aggressive 모드 레버리지 조건]
  - 3년 MA200 Gap < 45%: 레버리지 3.0x 적용
  - 45% <= 3년 MA200 Gap < 65%: 레버리지 2.5x 적용
  - 3년 MA200 Gap >= 65%: 레버리지 2.0x 적용
----------------------------

C:\Windows\Temp\ipykernel_19644\468671792.py:47: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


.... 완료.

현재 시장 상태: 정규장
실시간 가격: {'QQQ': '568.50', 'TQQQ': '89.95', 'PSQ': '32.85', 'SPY': '637.20'}
지표를 계산하는 중...

 마켓워치 분석 결과 (기준 시각: 2025-07-28 22:55:28 | 시장: 정규장)
250703-3-4

어제 포지션 (Normal): QQQ(75.0%) : TQQQ(25.0%)
오늘 포지션 (Normal): QQQ(75.0%) : TQQQ(25.0%)

주요 조건 달성 현황:
- SPY > MA200 (상승장 진입): O
- MA25 < SPY < MA200 (회복장 진입): X
- SPY < MA25 (하락장 지속): X
- SPY 전고점(MAX) 도달: X
--- 동적 할당 조건 ---
- Normal 모드 (1년전 QQQ MA200 대비 수익률: 34.32%)
   ㄴ 조건: >= 15% (레버리지 1.5x)
- Aggressive 모드 (3년전 SPY MA200 대비 Gap: 54.05%)
   ㄴ 조건: 45%-65% (레버리지 2.5x)

다음 분석을 위해 대기 중입니다...
프로그램을 종료합니다.
